In [ ]:
#title: amenities_osm.ipynb
#get amenities per district meeting certain criteria from openstreetmap
# first get the district relation ids from OSM
# then sort them in alphabetical order of name
# then for each district get the amenities of interest within the district boundary
# then filter by certain criteria (e.g. name contains certain keywords)

In [7]:
# %%
# get district relation ids of Andhra Pradesh (rel_id: 2022095) from OSM
#

import requests

# %%
# Define the Overpass API endpoint and query
overpass_url = "https://overpass-api.de/api/interpreter"

# Your Overpass Turbo query

# Compose QT query
query="""
[out:csv(::id, name)][timeout:60];
area(%d)->.searchArea;
(   relation["boundary"="administrative"]["admin_level"="5"](area.searchArea););
out body;

""" % (3600000000 +2022095)
# %%
# Send the request to Overpass API
response = requests.post(overpass_url, data={"data": query})

# Check for successful response
if response.status_code == 200:
    print("✅ Query executed successfully.")
else:
    raise Exception(f"Overpass API request failed with status code {response.status_code}")

# %%
# get the "response.content" into a pandas dataframe
import pandas as pd
from io import StringIO
df = pd.read_csv(StringIO(response.text), sep='\t')  


✅ Query executed successfully.


In [8]:
df.head()

,@id,name
0,2022096,Srikakulam district
1,2022097,Vizianagaram district
2,2022111,Visakhapatnam
3,2022126,East Godavari
4,2022127,West Godavari
